Imports

In [15]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import numpy as np

Device config

In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)

cpu


Parameters

In [17]:
block_size = 8
batch_size = 4
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250

Reading and text processing

In [18]:
filename = "crime_and_punishment.txt"
with open(filename, 'r', encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '0', '1', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'à', 'ä', 'æ', 'ç', 'è', 'é', 'ê', 'î', 'ï', 'ô', 'ö', 'ü', '‘', '’', '“', '”', '\ufeff']


Creation of encoding and decoding maps

In [19]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

Text encoding in tensors

In [20]:
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

Batch obtain function

In [21]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

Loss estimation function

In [22]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

Bigram language model definition

In [23]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
        
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

Model building and training

In [24]:
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


_öçmEQUH8*Q;)iK7pBPîä9ü t8.Goév[75bæZ‘IKjçàà4LtIzà5RèOa-x*éX“33Mv8.Vè*wïktyXrub.êée‘IUrho5kô]﻿:u“ àP(PcybwS
4-*IB*(F“dc[ê,qsL’dôlVlfü*érüPrKZru[f8(EF[h oy(rou9]t;vrF
1Cévu*E66‘ UBédçeê‘1,(xfgQ]ö﻿7ô_vKc?E‘öSîXïkDL“”vXHJ6n
3?q[:u’iéa’B16-””WD”;OönHàP4;(55dl5ZiI1:ruaüü1IAlKHçZSdLSCnQxfh[(yQWRGa[O-P.!h4e*-”mà?éT*U”J8*Mz7Fh8CJjYQxàskïruDU”!o;LyæV‘I,J0lfBo3!cQ)BVA;.yXDURBîe35üLpcoZcy.ïè?!iI(;;LGaàà*TRjpoJW-”MüP?9WTWä3“cwu1ôèàbBD94BE’4aRGêDscèêlfçhè-la?A;J]v”wêaIeR NJ“APtyôî0 TVjïpPb5VHRE7öHs’7ôrFaGéeî


Optimizer and training loop

In [25]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")

    xb, yb = get_batch('train')
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 4.937, val loss: 4.937
step: 250, train loss: 4.878, val loss: 4.865
step: 500, train loss: 4.800, val loss: 4.835
step: 750, train loss: 4.743, val loss: 4.758
4.870574474334717
